In [2]:
# install PyTorch Geometric for your torch version (GPU)
!pip install -q torch_geometric
!pip install -q torch_scatter torch_sparse torch_cluster torch_spline_conv \
  -f https://data.pyg.org/whl/torch-2.3.0+cu121.html


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 88.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.6/949.6 kB 8.1 MB/s eta 0:00:00


In [13]:
"""
GraphFEN (Heterogeneous Graph Financial Event Network)
Full end-to-end script with metric tracking: precision, recall, f1, loss, validation accuracy logged per epoch,
and final test metrics including test_loss, precision, recall, f1, accuracy, ROC AUC, PR AUC, confusion matrix.

Requirements: torch, numpy, pandas, scikit-learn, imblearn, torch_geometric
Adjust FILE_PATH and hyperparams as needed.
"""

import os
import hashlib
import random
import time
import pickle

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import BorderlineSMOTE

# PyG imports
try:
    from torch_geometric.data import HeteroData
    from torch_geometric.nn import HeteroConv, SAGEConv
except Exception as e:
    raise ImportError(
        "torch_geometric not available. Install PyG for your torch/cuda version. "
        "See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html") from e

from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, precision_recall_curve, auc as sklearn_auc,
                             confusion_matrix, classification_report)

In [14]:
# ---------------------------
# Hyperparams and device (tune these for speed / capacity)
# ---------------------------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

EPOCHS = 200
BATCH_SIZE = 2048       # increase if memory allows
EMBED_DIM = 64          # smaller for speed; increase for capacity
NUM_GNN_LAYERS = 1
PROJ_DIM = 64
LR = 1e-3

In [15]:
# ---------------------------
# Load & preprocess data
# ---------------------------
FILE_PATH = '/content/drive/MyDrive/credit card /creditcard.csv'  # change if needed
df = pd.read_csv(FILE_PATH)

# numeric columns only
df_numeric = df.select_dtypes(include=['number']).copy()
if 'Time' not in df_numeric.columns:
    df_numeric['Time'] = np.arange(len(df_numeric))

df_numeric = df_numeric.sort_values('Time').reset_index(drop=True)

y = df_numeric['Class'].astype(int).values
X = df_numeric.drop('Class', axis=1).values.astype(np.float32)
feature_names = df_numeric.drop('Class', axis=1).columns.tolist()

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# splits (stratified)
X_train, X_temp, y_train, y_temp, idx_train, idx_temp = train_test_split(
    X_scaled, y, np.arange(len(y)), test_size=0.2, random_state=SEED, shuffle=True, stratify=y
)
X_valid, X_test, y_valid, y_test, idx_valid, idx_test = train_test_split(
    X_temp, y_temp, idx_temp, test_size=0.5, random_state=SEED, shuffle=True, stratify=y_temp
)

# optional oversampling
sm = BorderlineSMOTE(random_state=SEED)
try:
    X_train_over, y_train_over = sm.fit_resample(X_train, y_train)
except Exception:
    X_train_over, y_train_over = X_train, y_train

X_all = X_scaled
y_all = y

In [16]:

# ---------------------------
# Synthetic IDs (replace with real IDs if available)
# ---------------------------
n_cards = 2000
n_merchants = 500
n_devices = 1500

def deterministic_hash_mod(val, mod):
    s = str(val).encode('utf-8')
    h = int(hashlib.md5(s).hexdigest()[:8], 16)
    return int(h % mod)

card_ids = np.array([deterministic_hash_mod(t, n_cards) for t in df_numeric['Time'].values])
amounts = df_numeric['Amount'].values
amount_bins = pd.qcut(amounts, q=min(50, len(amounts)//100+1), duplicates='drop').astype(str)
merchant_ids = np.array([deterministic_hash_mod(b, n_merchants) for b in amount_bins])
if all(c in df_numeric.columns for c in ['V1','V2','V3','V4']):
    device_basis = (df_numeric['V1'].astype(str) + '_' + df_numeric['V2'].astype(str) + '_' +
                    df_numeric['V3'].astype(str) + '_' + df_numeric['V4'].astype(str)).values
else:
    device_basis = df_numeric['Time'].astype(str).values
device_ids = np.array([deterministic_hash_mod(b, n_devices) for b in device_basis])

assert len(card_ids) == len(merchant_ids) == len(device_ids) == len(X_all)

In [17]:

# ---------------------------
# Build hetero graph
# ---------------------------
data = HeteroData()
data['card'].num_nodes = n_cards
data['merchant'].num_nodes = n_merchants
data['device'].num_nodes = n_devices

edge_index_card_merchant = np.vstack([card_ids, merchant_ids]).astype(np.int64)
edge_index_card_device = np.vstack([card_ids, device_ids]).astype(np.int64)

data['card', 'to_merchant', 'merchant'].edge_index = torch.tensor(edge_index_card_merchant, dtype=torch.long)
data['card', 'to_device', 'device'].edge_index = torch.tensor(edge_index_card_device, dtype=torch.long)

edge_attr_tx = torch.tensor(X_all, dtype=torch.float32)
edge_label = torch.tensor(y_all, dtype=torch.long)
data['card', 'to_merchant', 'merchant'].edge_attr = edge_attr_tx.clone()
data['card', 'to_merchant', 'merchant'].edge_label = edge_label.clone()
data['card', 'to_device', 'device'].edge_attr = edge_attr_tx.clone()
data['card', 'to_device', 'device'].edge_label = edge_label.clone()

# init node features by aggregating transaction means
data['card'].x = torch.zeros((n_cards, X_all.shape[1]), dtype=torch.float32)
data['merchant'].x = torch.zeros((n_merchants, X_all.shape[1]), dtype=torch.float32)
data['device'].x = torch.zeros((n_devices, X_all.shape[1]), dtype=torch.float32)

card_sums = np.zeros((n_cards, X_all.shape[1]), dtype=np.float32)
card_counts = np.zeros((n_cards,), dtype=np.int32)
for i, c in enumerate(card_ids):
    card_sums[c] += X_all[i]; card_counts[c] += 1
nonzero = card_counts > 0
data['card'].x[nonzero] = torch.tensor(card_sums[nonzero] / card_counts[nonzero, None], dtype=torch.float32)

merch_sums = np.zeros((n_merchants, X_all.shape[1]), dtype=np.float32)
merch_counts = np.zeros((n_merchants,), dtype=np.int32)
for i, m in enumerate(merchant_ids):
    merch_sums[m] += X_all[i]; merch_counts[m] += 1
nonzero = merch_counts > 0
data['merchant'].x[nonzero] = torch.tensor(merch_sums[nonzero] / merch_counts[nonzero, None], dtype=torch.float32)

dev_sums = np.zeros((n_devices, X_all.shape[1]), dtype=np.float32)
dev_counts = np.zeros((n_devices,), dtype=np.int32)
for i, d in enumerate(device_ids):
    dev_sums[d] += X_all[i]; dev_counts[d] += 1
nonzero = dev_counts > 0
data['device'].x[nonzero] = torch.tensor(dev_sums[nonzero] / dev_counts[nonzero, None], dtype=torch.float32)

In [18]:

# ---------------------------
# Model definitions
# ---------------------------
class HeteroEncoder(nn.Module):
    def __init__(self, in_channels, hidden_channels=EMBED_DIM, num_layers=NUM_GNN_LAYERS, out_channels=EMBED_DIM):
        super().__init__()
        self.forward_rels = [
            ('card', 'to_merchant', 'merchant'),
            ('card', 'to_device', 'device'),
        ]
        convs = []
        for layer in range(num_layers):
            in_c = in_channels if layer == 0 else hidden_channels
            conv_dict = {}
            for (src, rel, dst) in self.forward_rels:
                conv_dict[(src, rel, dst)] = SAGEConv(in_c, hidden_channels)
                rev_rel = f"rev_{rel}"
                conv_dict[(dst, rev_rel, src)] = SAGEConv(in_c, hidden_channels)
            convs.append(HeteroConv(conv_dict, aggr='mean'))
        self.convs = nn.ModuleList(convs)
        self.node_mlps = nn.ModuleDict({
            'card': nn.Linear(hidden_channels, out_channels),
            'merchant': nn.Linear(hidden_channels, out_channels),
            'device': nn.Linear(hidden_channels, out_channels),
        })

    def forward(self, x_dict, edge_index_dict, edge_attr_dict=None):
        h = {k: v for k, v in x_dict.items()}
        local_edge_index = {}
        for key, eidx in edge_index_dict.items():
            if not isinstance(eidx, torch.Tensor):
                eidx = torch.tensor(eidx, dtype=torch.long)
            if eidx.dim() != 2 or eidx.size(0) != 2:
                raise ValueError(f"edge_index for key {key} must be shape (2,E), got {tuple(eidx.shape)}")
            local_edge_index[key] = eidx
        for (src, rel, dst) in self.forward_rels:
            fkey = (src, rel, dst)
            if fkey not in local_edge_index:
                raise KeyError(f"Expected forward edge_index for {fkey} but it's missing.")
            eidx = local_edge_index[fkey]
            rev_key = (dst, f"rev_{rel}", src)
            if rev_key not in local_edge_index:
                local_edge_index[rev_key] = torch.stack([eidx[1], eidx[0]], dim=0)
        for conv in self.convs:
            h = conv(h, local_edge_index)
            for ntype, feat in h.items():
                h[ntype] = F.gelu(feat)
        out = {}
        for ntype in ['card', 'merchant', 'device']:
            if ntype not in h:
                out[ntype] = self.node_mlps[ntype](x_dict[ntype].to(next(iter(h.values())).device))
            else:
                out[ntype] = self.node_mlps[ntype](h[ntype])
        return out

class GraphFEN(nn.Module):
    def __init__(self, node_feature_dim, embed_dim=EMBED_DIM, proj_dim=PROJ_DIM, num_classes=2):
        super().__init__()
        self.encoder = HeteroEncoder(in_channels=node_feature_dim, hidden_channels=embed_dim,
                                     num_layers=NUM_GNN_LAYERS, out_channels=embed_dim)
        self.proj = nn.Sequential(nn.Linear(embed_dim*3, embed_dim), nn.ReLU(), nn.Linear(embed_dim, proj_dim))
        self.temporal_head = nn.Sequential(nn.Linear(embed_dim*3, embed_dim), nn.ReLU(), nn.Linear(embed_dim, node_feature_dim))
        self.classifier = nn.Sequential(nn.Linear(embed_dim*3 + node_feature_dim, embed_dim), nn.ReLU(), nn.Dropout(0.2), nn.Linear(embed_dim, num_classes))

    def forward_node_embed(self, data):
        x_dict = {ntype: data[ntype].x.to(DEVICE) for ntype in ['card','merchant','device']}
        edge_index_dict = {
            ('card','to_merchant','merchant'): data['card','to_merchant','merchant'].edge_index.to(DEVICE),
            ('card','to_device','device'): data['card','to_device','device'].edge_index.to(DEVICE),
        }
        node_emb = self.encoder(x_dict, edge_index_dict)
        return node_emb

    def classify_transaction_batch(self, node_emb, card_idx_batch, merchant_idx_batch, device_idx_batch, tx_feat_batch):
        card_e = node_emb['card'][card_idx_batch]; merch_e = node_emb['merchant'][merchant_idx_batch]; dev_e = node_emb['device'][device_idx_batch]
        cat = torch.cat([card_e, merch_e, dev_e], dim=1)
        logits = self.classifier(torch.cat([cat, tx_feat_batch], dim=1))
        return logits

# instantiate
node_feat_dim = X_all.shape[1]
model = GraphFEN(node_feature_dim=node_feat_dim, embed_dim=EMBED_DIM, proj_dim=PROJ_DIM, num_classes=2).to(DEVICE)

In [19]:

# ---------------------------
# Training utilities
# ---------------------------
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

all_tx_count = X_all.shape[0]
tx_indices = np.arange(all_tx_count)

def make_edge_batch(idx_batch):
    card_idx = torch.tensor(card_ids[idx_batch], dtype=torch.long, device=DEVICE)
    merchant_idx = torch.tensor(merchant_ids[idx_batch], dtype=torch.long, device=DEVICE)
    device_idx = torch.tensor(device_ids[idx_batch], dtype=torch.long, device=DEVICE)
    tx_feat = torch.tensor(X_all[idx_batch], dtype=torch.float32, device=DEVICE)
    labels = torch.tensor(y_all[idx_batch], dtype=torch.long, device=DEVICE)
    return card_idx, merchant_idx, device_idx, tx_feat, labels

def batched_eval(model, data, idx_list, batch_size=2048, device=DEVICE, compute_loss=True):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []
    total_loss = 0.0
    total_samples = 0

    data_device = HeteroData()
    data_device['card'].x = data['card'].x.to(device)
    data_device['merchant'].x = data['merchant'].x.to(device)
    data_device['device'].x = data['device'].x.to(device)
    data_device['card','to_merchant','merchant'].edge_index = data['card','to_merchant','merchant'].edge_index.to(device)
    data_device['card','to_device','device'].edge_index = data['card','to_device','device'].edge_index.to(device)

    with torch.no_grad():
        node_emb = model.forward_node_embed(data_device)

    n = len(idx_list)
    for start in range(0, n, batch_size):
        batch_idx = idx_list[start:start+batch_size]
        c_idx_b, m_idx_b, d_idx_b, tx_feat_b, labels_b = make_edge_batch(batch_idx)
        with torch.no_grad():
            logits = model.classify_transaction_batch(node_emb, c_idx_b, m_idx_b, d_idx_b, tx_feat_b)
            probs = F.softmax(logits, dim=1).cpu().numpy()
            preds = probs.argmax(axis=1)
            if compute_loss:
                loss = criterion(logits, labels_b)
                total_loss += loss.item() * labels_b.size(0)
                total_samples += labels_b.size(0)
        all_preds.append(preds); all_probs.append(probs); all_labels.append(labels_b.cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_probs = np.concatenate(all_probs)
    all_labels = np.concatenate(all_labels)

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, zero_division=0)
    rec = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)

    if len(np.unique(all_labels)) == 2:
        try:
            roc = roc_auc_score(all_labels, all_probs[:,1])
            p, r, _ = precision_recall_curve(all_labels, all_probs[:,1])
            pr_auc = sklearn_auc(r, p)
        except Exception:
            roc = float('nan'); pr_auc = float('nan')
    else:
        roc = float('nan'); pr_auc = float('nan')

    loss_avg = (total_loss / total_samples) if compute_loss and total_samples > 0 else float('nan')
    cm = confusion_matrix(all_labels, all_preds)
    return {'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1, 'roc': roc, 'pr_auc': pr_auc, 'cm': cm, 'loss': loss_avg}

/tmp/ipython-input-261004463.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


In [20]:

# ---------------------------
# Training loop with metric tracking
# ---------------------------
history = {'train_loss': [], 'val_acc': [], 'val_prec': [], 'val_rec': [], 'val_f1': [], 'val_roc': [], 'val_pr_auc': [], 'val_loss': []}
best_val_f1 = 0.0

for epoch in range(EPOCHS):
    t0 = time.time()
    model.train()

    # compute node embeddings once per epoch
    data_device = HeteroData()
    data_device['card'].x = data['card'].x.to(DEVICE)
    data_device['merchant'].x = data['merchant'].x.to(DEVICE)
    data_device['device'].x = data['device'].x.to(DEVICE)
    data_device['card','to_merchant','merchant'].edge_index = data['card','to_merchant','merchant'].edge_index.to(DEVICE)
    data_device['card','to_device','device'].edge_index = data['card','to_device','device'].edge_index.to(DEVICE)

    with torch.no_grad():
        node_emb = model.forward_node_embed(data_device)

    perm = np.random.permutation(tx_indices)
    running_loss = 0.0
    iters = 0
    pbar = tqdm(range(0, len(perm), BATCH_SIZE), desc=f"Epoch {epoch+1}/{EPOCHS}")
    for start in pbar:
        iters += 1
        batch_idx = perm[start:start+BATCH_SIZE]
        card_idx_b, merch_idx_b, dev_idx_b, tx_feat_b, labels_b = make_edge_batch(batch_idx)

        optimizer.zero_grad()
        with torch.amp.autocast("cuda", enabled=torch.cuda.is_available()):
            logits = model.classify_transaction_batch(node_emb, card_idx_b, merch_idx_b, dev_idx_b, tx_feat_b)
            loss = criterion(logits, labels_b)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        pbar.set_postfix({'loss': running_loss / iters})

    avg_train_loss = running_loss / max(1, iters)
    history['train_loss'].append(avg_train_loss)

    # validation (full)
    val_metrics = batched_eval(model, data, idx_valid, batch_size=BATCH_SIZE, device=DEVICE, compute_loss=True)
    history['val_acc'].append(val_metrics['acc'])
    history['val_prec'].append(val_metrics['prec'])
    history['val_rec'].append(val_metrics['rec'])
    history['val_f1'].append(val_metrics['f1'])
    history['val_roc'].append(val_metrics['roc'])
    history['val_pr_auc'].append(val_metrics['pr_auc'])
    history['val_loss'].append(val_metrics['loss'])

    epoch_time = time.time() - t0
    print(f"\nEpoch {epoch+1}/{EPOCHS} — time: {epoch_time:.1f}s — train_loss: {avg_train_loss:.6f}")
    print(f" Val -> acc: {val_metrics['acc']*100:.4f}%  prec: {val_metrics['prec']*100:.4f}%  rec: {val_metrics['rec']*100:.4f}%  f1: {val_metrics['f1']*100:.4f}%  roc: {val_metrics['roc']:.6f}  pr_auc: {val_metrics['pr_auc']:.6f}  val_loss: {val_metrics['loss']:.6f}")

    if val_metrics['f1'] > best_val_f1:
        best_val_f1 = val_metrics['f1']
        torch.save({'model_state_dict': model.state_dict()}, "best_graphfen_hetero.pth")
        print("Saved best model.")

Epoch 1/200:   0%|          | 0/140 [00:00<?, ?it/s]


Epoch 1/200 — time: 4.2s — train_loss: 0.061041
 Val -> acc: 99.8280%  prec: 0.0000%  rec: 0.0000%  f1: 0.0000%  roc: 0.904869  pr_auc: 0.565412  val_loss: 0.012112


Epoch 2/200:   0%|          | 0/140 [00:00<?, ?it/s]


Epoch 2/200 — time: 4.8s — train_loss: 0.012461
 Val -> acc: 99.8280%  prec: 0.0000%  rec: 0.0000%  f1: 0.0000%  roc: 0.946246  pr_auc: 0.699074  val_loss: 0.011154


Epoch 3/200:   0%|          | 0/140 [00:00<?, ?it/s]

...


Epoch 198/200 — time: 2.7s — train_loss: 0.003053
 Val -> acc: 99.9438%  prec: 85.1064%  rec: 81.6327%  f1: 83.3333%  roc: 0.972443  pr_auc: 0.763356  val_loss: 0.003192


Epoch 199/200:   0%|          | 0/140 [00:00<?, ?it/s]


Epoch 199/200 — time: 2.8s — train_loss: 0.003166
 Val -> acc: 99.9438%  prec: 85.1064%  rec: 81.6327%  f1: 83.3333%  roc: 0.972409  pr_auc: 0.762252  val_loss: 0.003255


Epoch 200/200:   0%|          | 0/140 [00:00<?, ?it/s]


Epoch 200/200 — time: 2.8s — train_loss: 0.003169
 Val -> acc: 99.9438%  prec: 85.1064%  rec: 81.6327%  f1: 83.3333%  roc: 0.972520  pr_auc: 0.762839  val_loss: 0.003244


In [21]:

# ---------------------------
# Final test evaluation (batched) + print detailed metrics
# ---------------------------
test_metrics = batched_eval(model, data, idx_test, batch_size=BATCH_SIZE, device=DEVICE, compute_loss=True)
print("\n===== Final Test results =====")
print(f"Test loss: {test_metrics['loss']:.6f}")
print(f"Acc: {test_metrics['acc']*100:.4f}%")
print(f"Precision: {test_metrics['prec']*100:.4f}%")
print(f"Recall: {test_metrics['rec']*100:.4f}%")
print(f"F1: {test_metrics['f1']*100:.4f}%")
print(f"ROC AUC: {test_metrics['roc']:.6f}")
print(f"PR AUC: {test_metrics['pr_auc']:.6f}")
print("Confusion matrix:\n", test_metrics['cm'])

# classification report
y_true_sample = []
y_pred_sample = []
# produce full preds and labels to show classification report (reuse batched_eval internals if needed)
# Here we compute again with compute_loss=False but collect preds/labels
def collect_preds(model, data, idx_list, batch_size=2048, device=DEVICE):
    model.eval()
    all_preds, all_labels = [], []
    data_device = HeteroData()
    data_device['card'].x = data['card'].x.to(device)
    data_device['merchant'].x = data['merchant'].x.to(device)
    data_device['device'].x = data['device'].x.to(device)
    data_device['card','to_merchant','merchant'].edge_index = data['card','to_merchant','merchant'].edge_index.to(device)
    data_device['card','to_device','device'].edge_index = data['card','to_device','device'].edge_index.to(device)
    with torch.no_grad():
        node_emb = model.forward_node_embed(data_device)
    n = len(idx_list)
    for start in range(0, n, batch_size):
        batch_idx = idx_list[start:start+batch_size]
        c_idx_b, m_idx_b, d_idx_b, tx_feat_b, labels_b = make_edge_batch(batch_idx)
        with torch.no_grad():
            logits = model.classify_transaction_batch(node_emb, c_idx_b, m_idx_b, d_idx_b, tx_feat_b)
            probs = F.softmax(logits, dim=1).cpu().numpy()
            preds = probs.argmax(axis=1)
        all_preds.append(preds)
        all_labels.append(labels_b.cpu().numpy())
    return np.concatenate(all_preds), np.concatenate(all_labels)

preds_all, labels_all = collect_preds(model, data, idx_test, batch_size=BATCH_SIZE, device=DEVICE)
print("\nClassification report (test):\n", classification_report(labels_all, preds_all, digits=4, zero_division=0))


===== Final Test results =====
Test loss: 0.002695
Acc: 99.9298%
Precision: 80.8511%
Recall: 77.5510%
F1: 79.1667%
ROC AUC: 0.981658
PR AUC: 0.824749
Confusion matrix:
 [[28423     9]
 [   11    38]]

Classification report (test):
               precision    recall  f1-score   support

           0     0.9996    0.9997    0.9996     28432
           1     0.8085    0.7755    0.7917        49

    accuracy                         0.9993     28481
   macro avg     0.9041    0.8876    0.8957     28481
weighted avg     0.9993    0.9993    0.9993     28481



In [22]:

# ---------------------------
# Save final model & artifacts
# ---------------------------
torch.save({'model_state_dict': model.state_dict()}, "best_graphfen_hetero.pth")
with open("graphfen_artifacts.pkl", "wb") as f:
    pickle.dump({
        'scaler': scaler if 'scaler' in globals() else None,
        'card_ids': card_ids,
        'merchant_ids': merchant_ids,
        'device_ids': device_ids,
        'feature_names': feature_names,
        'history': history
    }, f)

print("\nSaved model to best_graphfen_hetero.pth and artifacts to graphfen_artifacts.pkl")


Saved model to best_graphfen_hetero.pth and artifacts to graphfen_artifacts.pkl
